# 二元操作记录器

二元操作记录器用于记录用户是否执行了指定的操作，如，实现签到功能，判断用户某一天是否完成了签到。如果单纯考虑存储用户的签到记录，只要使用集合或者列表存储用户每次签到的日期即可。但这样一来，对用户数量庞大的网站来说，实现这个简单的功能需要耗费大量内存。

另外，虽然使用 HyperLogLog 能够以极低的内存记录大量数据，但它的概率特性决定了它无法用于准确记录的场景，比如要判断一个员工的出勤情况。




可以使用 Redis 的位图数据结构，它可以存储连续的一串二进制位，并基于索引对各个二进制位进行设置。对于位图数据结构可以简单看一下这里的讲解，[《BitMap》](https://www.xiaolincoding.com/redis/data_struct/command.html#bitmap)。
> 位图（Bitmap）是 Redis 中的一种高效数据结构，使用二进制位存储数据，每个位表示一个布尔值（0 或 1），以整数偏移量为索引。它的核心是通过位操作（如 SETBIT、GETBIT、BITOP）实现数据的设置、查询和逻辑运算，占用极少内存，适合大规模布尔数据场景，如用户在线状态、签到记录或去重统计。例如，1000 万用户的状态只需约 1.25MB（1000 万位）。位图支持快速的位级运算（如与、或、异或），但仅限于二值数据，功能较单一，适合高密度、高性能的特定场景。

怎么用呢？比如可以以 2025 年 1 月 1 日作为基准日起，那么位图索引 0 上的二进制位会记录 2025 年 1 月 1 日的签到情况，位图索引 1 上的二进制位会记录 2025 年 1 月 2 日的签到情况。在这个基础上，如果想检查该用户的全勤情况，只需要检查指定索引范围内的全部二进制位即可。

在不计算 Redis 存储数据所需额外消耗的情况下，单个用户的每次签到情况只需占用内存的一个二进制位，而记录一年签到所需的内存也仅为 365 位。即使保存 10 年的签到记录，单个用户所需的内存也不过是 3650 位，约 0.45KB。这样的低内存占用完全能够满足多用户、长时间跨度的记录要求。

In [ ]:
class BinaryRecorder:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def setbit(self, index):
        """
        将指定索引上的二进制位设置为1。
        """
        # SETBIT key index 1
        self.client.setbit(self.key, index, 1)

    def clearbit(self, index):
        """
        将指定索引上的二进制位设置为0。
        """
        # SETBIT key index 0
        self.client.setbit(self.key, index, 0)

    def getbit(self, index):
        """
        获取指定索引上的二进制位的值。
        """
        # GETBIT key index
        return self.client.getbit(self.key, index)

    def countbits(self, start, end):
        """
        统计指定索引区间内，值为1的二进制位数量。
        """
        # BITCOUNT key start end
        return self.client.bitcount(self.key, start, end, "BIT")

真会拿 Redis 保存一年或多年的签到数据吗？

MySQL 主要问题有：
+ 占用空间大，每条签到记录（用户ID+日期）需几十字节，千万用户 10 年约需数百 GB；Redis 位图仅 0.45KB/用户，约 4.5GB。
+ 速度慢。

而且 Redis 有 RDB 和 AOF 也不太会丢数据。我觉得最终可能的方案是，Redis 可以存近 1-3 年的“热数据”，到期后对这些“冷数据”，可以按（user_id | year | BitMap），结合压缩方案，归档到 MySQL 中。